In [56]:
import os
import sys
import cv2
import csv
import tqdm
import pandas as pd
import random
import numpy as np
import time
from skimage.morphology import skeletonize, medial_axis
from matplotlib import pyplot as plt

In [57]:

def bb_intersection_over_union(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    # compute the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)
    # return the intersection over union value
    return iou

In [58]:
CSV_SORT_PATH = "C:/Users/benja\Desktop/NN_alive_dead_training_vids/Sorted/356_sort_forward.csv"
df = pd.read_csv(CSV_SORT_PATH,names=('frame', 'x1', 'y1', 'x2', 'y2','label','delta'))
df['catagory'] = 'alive'
#df['X']=df[['x1','x2']].mean(axis=1)
#df['Y']=df[['y1','y2']].mean(axis=1)
#df = df[['frame', 'X', 'Y','label']]
uniqueFrames = df["frame"].unique()
uniqueIDs = df["label"].unique()

In [59]:
threshold = 45
vc = df.label.value_counts()
test = vc[vc > threshold].index.tolist()
csv_outputs = []
deadboxes = []
for ID in test:        
    filtval = df['label'] ==ID
    interim = df[filtval]
    interimD = []
    interim2 = np.asarray(interim)
    fill = 0
    deadcount = 0
    for row in interim2:
        frameNA, x1A, y1A, x2A, y2A, labelA, deltaA, catagoryA, *_ = row
        if fill > 1:
            boxA = [x1A, y1A, x2A, y2A]
            boxB = [x1B, y1B, x2B, y2B]
            deltaA = bb_intersection_over_union(boxA, boxB) 
            #print(deltaA)
            if deltaA > 0.95:
                deadcount += 1
                #print(deadcount)
            #print(fill)
            if deadcount > 10:
                catagoryA = 'dead'
               # print(deadcount)
            if deadcount == 11:
                if deadboxes == []:
                    deadboxes.append([x1A, y1A, x2A, y2A])               
                    csv_outputs.append(frameNA)
                else:
                    isunique = 0
                    for box in deadboxes:
                        #print(box)
                        x1D, y1D, x2D, y2D, *_ = box
                        boxD = [x1D, y1D, x2D, y2D]
                        deltaD = bb_intersection_over_union(boxA, boxD)  
                        if deltaD > 0.8:
                            isunique = 1
                    if isunique == 0:
                        deadboxes.append([x1A, y1A, x2A, y2A])               
                        csv_outputs.append(int(frameNA))

                #print(deathtime)
                #print(frameNA)
            #newRow = [frameNA, x1A, y1A, x2A, y2A, labelA, deltaA, catagoryA]
            #interimD.append(newRow)
        frameNB, x1B, y1B, x2B, y2B,labelB, deltaB, catagoryB, *_ = row
        fill +=1
        #interimD.append(deathtime)
    #interimD = pd.DataFrame(interimD, columns = ['deathframe'])
    #csv_outputs.append(interimD)
#large_df = pd.concat(csv_outputs, ignore_index=True)  
csv_outputs = pd.DataFrame(csv_outputs, columns = ['deathframe'])
OUT_PATH = 'C:/Users/benja/Desktop/NN_alive_dead_training_vids/Sorted_Deathtimes/356_sorted_AD.csv'
pd.DataFrame(csv_outputs).to_csv(OUT_PATH, mode='a', header=False, index=None)

In [36]:
print(deadboxes)

[[581.492500976467, 624.4790838098402, 647.0509882827544, 643.7174491929782], [501.34910675634296, 405.2739349768932, 536.6417227306088, 452.188582408094], [680.1955755671454, 571.7159143243024, 701.9740311326893, 615.2931380820288], [727.5135094250129, 503.1226790457655, 770.228918438874, 553.7532969025741], [527.1301332046196, 262.9232818910121, 586.336043128546, 278.3732654573447], [530.7501316106896, 264.37954652886714, 588.2348561330359, 279.0954903616965], [631.3855639723081, 91.28704643417186, 697.6484745141461, 119.98622212331452], [955.2986630910998, 826.8915620566629, 1003.1524350962908, 860.7929487200886], [959.6389953395222, 828.1548133771506, 1007.4841319744944, 861.3773312640559], [881.782656606674, 413.4892135722458, 913.0587996820784, 466.92714065057993], [894.4503588039829, 524.3359517763527, 932.689920605044, 564.7429368952576], [990.40436995862, 247.7365398925408, 1056.456675432852, 269.77214641614967], [1067.0691549037558, 702.919805227893, 1132.2840491504187, 731.8

In [39]:
print(csv_outputs)

[441.0, 498.0, 627.0, 609.0, 587.0, 595.0, 682.0, 678.0, 682.0, 704.0, 771.0, 770.0, 855.0, 695.0, 526.0, 1015.0, 1016.0, 478.0]
